In [20]:
from google.cloud import bigquery
import pandas 

In [7]:
# Create a 'Client' object
client = bigquery.Client()

Using Kaggle's public dataset BigQuery integration.


We'll work with a dataset of posts on Hacker News, a website focusing on computer science and cybersecurity news. 

In [8]:
# Construct a reference to the "hacker_news" dataset
dataset_ref = client.dataset("hacker_news", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

How many tables are in the Hacker News dataset?

In [9]:
# List all the tables in the "hacker_news" dataset
tables = list(client.list_tables(dataset))

# Print names of all tables in the dataset 
for table in tables:  
    print(table.table_id)

full


How many columns in the 'full' table have 'STRING' data?

In [10]:
# Construct a reference to the "full" table
table_ref = dataset_ref.table("full")

# API request - fetch the table
table = client.get_table(table_ref)

In [11]:
# Print information on all the columns in the "full" table in the "hacker_news" dataset
table.schema

[SchemaField('title', 'STRING', 'NULLABLE', 'Story title', (), None),
 SchemaField('url', 'STRING', 'NULLABLE', 'Story url', (), None),
 SchemaField('text', 'STRING', 'NULLABLE', 'Story or comment text', (), None),
 SchemaField('dead', 'BOOLEAN', 'NULLABLE', 'Is dead?', (), None),
 SchemaField('by', 'STRING', 'NULLABLE', "The username of the item's author.", (), None),
 SchemaField('score', 'INTEGER', 'NULLABLE', 'Story score', (), None),
 SchemaField('time', 'INTEGER', 'NULLABLE', 'Unix time', (), None),
 SchemaField('timestamp', 'TIMESTAMP', 'NULLABLE', 'Timestamp for the unix time', (), None),
 SchemaField('type', 'STRING', 'NULLABLE', 'type of details (comment comment_ranking poll story job pollopt)', (), None),
 SchemaField('id', 'INTEGER', 'NULLABLE', "The item's unique id.", (), None),
 SchemaField('parent', 'INTEGER', 'NULLABLE', 'Parent comment ID', (), None),
 SchemaField('descendants', 'INTEGER', 'NULLABLE', 'Number of story or poll descendants', (), None),
 SchemaField('ran

In [12]:
# Preview the first five lines of the "full" table
client.list_rows(table, max_results=5).to_dataframe()

,title,url,text,dead,by,score,time,timestamp,type,id,parent,descendants,ranking,deleted
0,None,None,"If the crocodile looked him up on Google, we b...",None,raxxorrax,NaN,1633421535,2021-10-05 08:12:15+00:00,comment,28756662,28750122,NaN,NaN,None
1,None,None,What exactly are you looking for? I think Pyto...,None,abiro,NaN,1569141387,2019-09-22 08:36:27+00:00,comment,21040311,21040141,NaN,NaN,None
2,None,None,"Ironically, this very project might help out w...",None,mjevans,NaN,1505769703,2017-09-18 21:21:43+00:00,comment,15279716,15276626,NaN,NaN,None
3,None,None,As you start to gain some experience it can be...,None,every_other,NaN,1538575027,2018-10-03 13:57:07+00:00,comment,18130207,18128477,NaN,NaN,None
4,None,None,"That’s what I was referring to, yes. I heard o...",None,manmal,NaN,1615664155,2021-03-13 19:35:55+00:00,comment,26449260,26449237,NaN,NaN,None


In [13]:
# Preview the first five entries in the "by" column of the "full" table
client.list_rows(table, selected_fields=table.schema[:1], max_results=5).to_dataframe()

,title
0,None
1,None
2,None
3,None
4,None


In [31]:
# Query to select comments that received more than 1000 replies
query_popular = """
                SELECT parent, COUNT(id)
                FROM `bigquery-public-data.hacker_news.full`
                GROUP BY parent
                HAVING COUNT(id) > 1000
                """


In [32]:
# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 10 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query_popular, job_config=safe_config)

# API request - run the query, and convert the results to a pandas DataFrame
popular_comments = query_job.to_dataframe()

# Print the first five rows of the DataFrame
popular_comments.head()

,parent,f0_
0,NaN,4189206
1,363.0,1184


**Aliasing and some more improvements**

In [35]:
# Query to select comments that received more than 1000 replies
query_improved = """
                SELECT parent, COUNT(1) AS Num_Posts
                FROM `bigquery-public-data.hacker_news.full`
                GROUP BY parent
                HAVING COUNT(1) > 1000
                """
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query_improved, job_config=safe_config)

# API request - run the query, and convert the results to a pandas DataFrame
improved_df = query_job.to_dataframe()

# Print the first five rows of the DataFrame
improved_df.head()

,parent,Num_Posts
0,363.0,1184
1,NaN,4189206


We want to award the authors that have more than 10000 posts.

In [39]:
# Query to select prolific commenters and post counts
prolific_commenters_query = """
                            SELECT `by` AS author, COUNT(1) AS NumPosts 
                            FROM `bigquery-public-data.hacker_news.full`
                            GROUP BY author
                            HAVING COUNT(1) > 10000 
                            """ 

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(prolific_commenters_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
prolific_commenters = query_job.to_dataframe()

# View top few rows of results
print(prolific_commenters.head())



        author  NumPosts
0        ghaff     17654
1   stcredzero     15175
2  chrisseaton     12159
3   walterbell     10914
4          lmm     16189
